# CNN_10channel

Abstract:
- single channel: band_avg
- CNN, small net

Result:
- Kaggle score: 

References:
- https://www.kaggle.com/ivalmian/simple-svd-xgboost-baseline-lb-35
- https://www.kaggle.com/arieltci/a-keras-prototype-0-21174-on-pl

## 1. Preprocess

### Import pkgs

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss, accuracy_score
from IPython.display import display

%matplotlib inline

In [2]:
import os
import time
import zipfile
import lzma
import pickle
from PIL import Image
from shutil import copy2

### Run name

In [3]:
project_name = 'SC_Iceberg_Classifier'
step_name = 'CNN_4channel'
date_str = time.strftime("%Y%m%d", time.localtime())
time_str = time.strftime("%Y%m%d_%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)

run_name: SC_Iceberg_Classifier_CNN_4channel_20171107_162243


### Basic folders

In [4]:
cwd = os.getcwd()
input_path = os.path.join(cwd, 'input')
log_path = os.path.join(cwd, 'log')
model_path = os.path.join(cwd, 'model')
output_path = os.path.join(cwd, 'output')
print('input_path: ' + input_path)
print('log_path: ' + log_path)
print('model_path: ' + model_path)
print('output_path: ' + output_path)

input_path: E:\Kaggle\Statoil_C_CORE_Iceberg_Classifier_Challenge\input
log_path: E:\Kaggle\Statoil_C_CORE_Iceberg_Classifier_Challenge\log
model_path: E:\Kaggle\Statoil_C_CORE_Iceberg_Classifier_Challenge\model
output_path: E:\Kaggle\Statoil_C_CORE_Iceberg_Classifier_Challenge\output


### Load data

In [5]:
sample_submission_path = os.path.join(input_path, 'sample_submission.csv')
sample_submission = pd.read_csv(sample_submission_path)
print(sample_submission.shape)
sample_submission.head(2)

(8424, 2)


,id,is_iceberg
0,5941774d,0.5
1,4023181e,0.5


In [6]:
is_iceberg_path = os.path.join(input_path, 'is_iceberg.p')

y_data = pickle.load(open(is_iceberg_path, mode='rb'))

print(y_data.shape)

(1604,)


In [7]:
%%time
#Load orignal data
inc_angle_data_path = os.path.join(input_path, 'inc_angle_data.p')
inc_angle_test_path = os.path.join(input_path, 'inc_angle_test.p')

inc_angle_data = pickle.load(open(inc_angle_data_path, mode='rb'))
inc_angle_test = pickle.load(open(inc_angle_test_path, mode='rb'))

print(inc_angle_data.shape)
print(inc_angle_test.shape)

(1604,)
(8424,)
Wall time: 31.4 ms


In [8]:
%%time
#Load orignal data
band1_data_path = os.path.join(input_path, 'band1_data.p')
band2_data_path = os.path.join(input_path, 'band2_data.p')
band_avg_data_path = os.path.join(input_path, 'band_avg_data.p')
band1_test_path = os.path.join(input_path, 'band1_test.p')
band2_test_path = os.path.join(input_path, 'band2_test.p')
band_avg_test_path = os.path.join(input_path, 'band_avg_test.p')

band1_data = pickle.load(open(band1_data_path, mode='rb'))
band2_data = pickle.load(open(band2_data_path, mode='rb'))
band_avg_data = pickle.load(open(band_avg_data_path, mode='rb'))
band1_test = pickle.load(open(band1_test_path, mode='rb'))
band2_test = pickle.load(open(band2_test_path, mode='rb'))
band_avg_test = pickle.load(open(band_avg_test_path, mode='rb'))

print(band1_data.shape)
print(band2_data.shape)
print(band_avg_data.shape)
print(band1_test.shape)
print(band2_test.shape)
print(band_avg_test.shape)

(1604, 75, 75)
(1604, 75, 75)
(1604, 75, 75)
(8424, 75, 75)
(8424, 75, 75)
(8424, 75, 75)
Wall time: 12.7 s


In [9]:
%%time
#Load orignal data
band1_data_edges_path = os.path.join(input_path, 'band1_data_edges.p')
band2_data_edges_path = os.path.join(input_path, 'band2_data_edges.p')
band_avg_data_edges_path = os.path.join(input_path, 'band_avg_data_edges.p')
band1_test_edges_path = os.path.join(input_path, 'band1_test_edges.p')
band2_test_edges_path = os.path.join(input_path, 'band2_test_edges.p')
band_avg_test_edges_path = os.path.join(input_path, 'band_avg_test_edges.p')

band1_data_edges = pickle.load(open(band1_data_edges_path, mode='rb'))
band2_data_edges = pickle.load(open(band2_data_edges_path, mode='rb'))
band_avg_data_edges = pickle.load(open(band_avg_data_edges_path, mode='rb'))
band1_test_edges = pickle.load(open(band1_test_edges_path, mode='rb'))
band2_test_edges = pickle.load(open(band2_test_edges_path, mode='rb'))
band_avg_test_edges = pickle.load(open(band_avg_test_edges_path, mode='rb'))

print(band1_data_edges.shape)
print(band2_data_edges.shape)
print(band_avg_data_edges.shape)
print(band1_test_edges.shape)
print(band2_test_edges.shape)
print(band_avg_test_edges.shape)

(1604, 75, 75)
(1604, 75, 75)
(1604, 75, 75)
(8424, 75, 75)
(8424, 75, 75)
(8424, 75, 75)
Wall time: 12.1 s


In [10]:
%%time
#Load orignal data
band1_data_gabor_path = os.path.join(input_path, 'band1_data_gabor.p')
band2_data_gabor_path = os.path.join(input_path, 'band2_data_gabor.p')
band_avg_data_gabor_path = os.path.join(input_path, 'band_avg_data_gabor.p')
band1_test_gabor_path = os.path.join(input_path, 'band1_test_gabor.p')
band2_test_gabor_path = os.path.join(input_path, 'band2_test_gabor.p')
band_avg_test_gabor_path = os.path.join(input_path, 'band_avg_test_gabor.p')

band1_data_gabor = pickle.load(open(band1_data_gabor_path, mode='rb'))
band2_data_gabor = pickle.load(open(band2_data_gabor_path, mode='rb'))
band_avg_data_gabor = pickle.load(open(band_avg_data_gabor_path, mode='rb'))
band1_test_gabor = pickle.load(open(band1_test_gabor_path, mode='rb'))
band2_test_gabor = pickle.load(open(band2_test_gabor_path, mode='rb'))
band_avg_test_gabor = pickle.load(open(band_avg_test_gabor_path, mode='rb'))

print(band1_data_gabor.shape)
print(band2_data_gabor.shape)
print(band_avg_data_gabor.shape)
print(band1_test_gabor.shape)
print(band2_test_gabor.shape)
print(band_avg_test_gabor.shape)

(1604, 75, 75)
(1604, 75, 75)
(1604, 75, 75)
(8424, 75, 75)
(8424, 75, 75)
(8424, 75, 75)
Wall time: 13 s


In [11]:
%%time
x_data = np.concatenate([band1_data[:, :, :, np.newaxis],
                         band2_data[:, :, :, np.newaxis],
                         band_avg_data[:, :, :, np.newaxis]], axis=-1)
print(x_data.shape)
x_test = np.concatenate([band1_test[:, :, :, np.newaxis],
                         band2_test[:, :, :, np.newaxis],
                         band_avg_test[:, :, :, np.newaxis],], axis=-1)
print(x_test.shape)

(1604, 75, 75, 3)
(8424, 75, 75, 3)
Wall time: 1.24 s


In [12]:
# %%time
# x_data = np.concatenate([band1_data[:, :, :, np.newaxis],
#                          band2_data[:, :, :, np.newaxis],
#                          band_avg_data[:, :, :, np.newaxis],
#                          band1_data_edges[:, :, :, np.newaxis],
#                          band2_data_edges[:, :, :, np.newaxis],
#                          band_avg_data_edges[:, :, :, np.newaxis],
#                          band1_data_gabor[:, :, :, np.newaxis],
#                          band2_data_gabor[:, :, :, np.newaxis],
#                          band_avg_data_gabor[:, :, :, np.newaxis]], axis=-1)
# print(x_data.shape)
# x_test = np.concatenate([band1_test[:, :, :, np.newaxis],
#                          band2_test[:, :, :, np.newaxis],
#                          band_avg_test[:, :, :, np.newaxis],
#                          band1_test_edges[:, :, :, np.newaxis],
#                          band2_test_edges[:, :, :, np.newaxis],
#                          band_avg_test_edges[:, :, :, np.newaxis],
#                          band1_test_gabor[:, :, :, np.newaxis],
#                          band2_test_gabor[:, :, :, np.newaxis],
#                          band_avg_test_gabor[:, :, :, np.newaxis]], axis=-1)
# print(x_test.shape)

In [13]:
%%time
x_train, x_val, inc_angle_train, inc_angle_val, y_train, y_val = train_test_split(x_data, inc_angle_data, y_data, test_size=0.15, shuffle=True, random_state=31)
print(x_train.shape)
print(x_val.shape)
print(inc_angle_train.shape)
print(inc_angle_val.shape)
print(y_train.shape)
print(y_val.shape)

(1363, 75, 75, 3)
(241, 75, 75, 3)
(1363,)
(241,)
(1363,)
(241,)
Wall time: 189 ms


## 2. Build model

In [14]:
from keras.utils.np_utils import to_categorical # convert to one-hot-encoding
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalMaxPooling2D, BatchNormalization, Input
from keras.layers.merge import Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, TensorBoard

Using TensorFlow backend.


In [15]:
def build_model():
    model = Sequential()

    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu',
                     input_shape = (75, 75, 3)))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 128, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(strides=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(filters = 256, kernel_size = (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units = 1, activation = 'sigmoid'))

    model.compile(optimizer = Adam(lr=1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [16]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 64)        1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 73, 73, 64)        256       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        36928     
_________________________________________________________________
batch_normalization_2 (Batch (None, 71, 71, 64)        256       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 128)       73856     
__________

In [17]:
def saveModel(model, run_name):
    cwd = os.getcwd()
    modelPath = os.path.join(cwd, 'model')
    if not os.path.isdir(modelPath):
        os.mkdir(modelPath)
    weigthsFile = os.path.join(modelPath, run_name + '.h5')
    model.save(weigthsFile)
saveModel(model, 'saveModel_test')

In [18]:
def get_lr(x):
    lr = round(1e-4 * 0.98 ** x, 6)
    if lr < 5e-5:
        lr = 5e-5
    print(lr, end='  ')
    return lr

# annealer = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)
annealer = LearningRateScheduler(get_lr)

log_dir = os.path.join(log_path, run_name)
print('log_dir:' + log_dir)
tensorBoard = TensorBoard(log_dir=log_dir)

log_dir:E:\Kaggle\Statoil_C_CORE_Iceberg_Classifier_Challenge\log\SC_Iceberg_Classifier_CNN_4channel_20171107_162243


In [ ]:
datagen = ImageDataGenerator(
    rotation_range=0,
    width_shift_range=0,
    height_shift_range=0,
    horizontal_flip=False,
    vertical_flip=False)

In [ ]:
%%time
hist = model.fit_generator(datagen.flow(x_train, y_train, batch_size=8, shuffle = True),
                           steps_per_epoch=len(x_train) / 8, 
                           epochs = 200, #1 for ETA, 0 for silent
                           verbose= 1,
                           max_queue_size= 16, 
                           workers= 8,
                           validation_data=(x_val, y_val),
                           callbacks=[annealer, tensorBoard])
# hist = model.fit_generator([x_train, inc_angle_train], y_train, 
#                  batch_size = 8, 
#                  verbose= 1,
#                  epochs = 30, #1 for ETA, 0 for silent
#                  validation_data=([x_val, inc_angle_val], y_val),
#                  callbacks=[tensorBoard])

0.0001  Epoch 1/200
171/170 [==============================] - 19s - loss: 0.9205 - acc: 0.6019 - val_loss: 0.7296 - val_acc: 0.4523
9.8e-05  Epoch 2/200
171/170 [==============================] - 17s - loss: 0.6570 - acc: 0.6718 - val_loss: 0.6258 - val_acc: 0.6722
9.6e-05  Epoch 3/200
171/170 [==============================] - 17s - loss: 0.5078 - acc: 0.7444 - val_loss: 0.6047 - val_acc: 0.6763
9.4e-05  Epoch 4/200
171/170 [==============================] - 17s - loss: 0.4491 - acc: 0.7992 - val_loss: 0.5479 - val_acc: 0.7427 ETA: 3s - loss: 0.4701 - acc: 0.7 - ETA: 
9.2e-05  Epoch 5/200
171/170 [==============================] - 17s - loss: 0.3945 - acc: 0.8360 - val_loss: 0.4102 - val_acc: 0.8216
9e-05  Epoch 6/200
171/170 [==============================] - 17s - loss: 0.3593 - acc: 0.8494 - val_loss: 0.3473 - val_acc: 0.8216
8.9e-05  Epoch 7/200
171/170 [==============================] - 17s - loss: 0.3155 - acc: 0.8655 - val_loss: 0.3739 - val_acc: 0.8299
8.7e-05  Epoch 8/200
17

In [ ]:
final_loss, final_acc = model.evaluate(x_val, y_val, verbose=1)
print("Final loss: {0:.4f}, final accuracy: {1:.4f}".format(final_loss, final_acc))

In [ ]:
val_prob1 = model.predict(x_val)

# print('Val log_loss: {}'.format(log_loss(y_val, val_prob1)))
val_prob1_limit = np.clip(val_prob1, 0.00005, 0.99995)
loss = log_loss(y_val, val_prob1_limit)
print('Val log_loss: {}'.format(loss))

val_prob1_limit = np.clip(val_prob1_limit, 0.05, 0.95)
loss = log_loss(y_val, val_prob1_limit)
print('Val log_loss: {}'.format(loss))

In [ ]:
final_acc_str = str(int(loss*10000))
run_name_acc = project_name + '_' + step_name + '_' + time_str + '_' + final_acc_str
print(run_name_acc)

In [ ]:
histories = pd.DataFrame(hist.history)
histories['epoch'] = hist.epoch
print(histories.columns)
histories_file = os.path.join(model_path, run_name_acc + '.csv')
histories.to_csv(histories_file, index=False)

In [ ]:
plt.plot(histories['loss'], color='b')
plt.plot(histories['val_loss'], color='r')
plt.show()
plt.plot(histories['acc'], color='b')
plt.plot(histories['val_acc'], color='r')
plt.show()

In [ ]:
saveModel(model, run_name_acc)

## 3. Predict

In [ ]:
if not os.path.exists(output_path):
    os.mkdir(output_path)
pred_file = os.path.join(output_path, run_name_acc + '.csv')
print(pred_file)

In [ ]:
test_prob = model.predict(x_test)
print(test_prob.shape)
print(test_prob[0:2])
test_prob = np.clip(test_prob, 0.05, 0.95)
print(test_prob.shape)
print(test_prob[0:2])

In [ ]:
sample_submission['is_iceberg'] = test_prob
print(sample_submission[0:2])
print(sample_submission.shape)
sample_submission.to_csv(pred_file, index=False)

In [ ]:
print(run_name_acc)
print('Done!')